#### 미세조정 전 후 비교

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification


test_reviews = [
    "This movie is absolutely amazing! Best film I've ever seen.",
    "Terrible waste of time. Worst movie ever made.",
    "It was okay, nothing special but not bad either.",
    "Brilliant acting and stunning visuals throughout!",
    "Boring and predictable plot. Very disappointed."
]

labels = ["부정", "긍정"]

# 미세 조정전 모델(랜덤 초기화)
BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
model_before = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME)
model_before.eval()
with torch.no_grad():
    for i ,review in enumerate(test_reviews):
        inputs = tokenizer(review, return_tensors='pt')
        logits = model_before(**inputs).logits
        probs = torch.softmax(logits, dim=-1)[0]
        pred_label = logits.argmax().item()

        print(f'리뷰: {i+1} : {review}')
        print(f'예측 : {labels[pred_label]} {probs[pred_label]:.4f}')
        print(f'긍정 확률 : {probs[1]:.4f}, 부정 확률 : {probs[0]:.4f}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


리뷰: 1 : This movie is absolutely amazing! Best film I've ever seen.
예측 : 긍정 0.6370
긍정 확률 : 0.6370, 부정 확률 : 0.3630
리뷰: 2 : Terrible waste of time. Worst movie ever made.
예측 : 긍정 0.6137
긍정 확률 : 0.6137, 부정 확률 : 0.3863
리뷰: 3 : It was okay, nothing special but not bad either.
예측 : 긍정 0.5955
긍정 확률 : 0.5955, 부정 확률 : 0.4045
리뷰: 4 : Brilliant acting and stunning visuals throughout!
예측 : 긍정 0.6267
긍정 확률 : 0.6267, 부정 확률 : 0.3733
리뷰: 5 : Boring and predictable plot. Very disappointed.
예측 : 긍정 0.6133
긍정 확률 : 0.6133, 부정 확률 : 0.3867


In [9]:
train_texts = [
    ("This is wonderful and fantastic!", 1),  # 긍정
    ("Absolutely terrible and awful!", 0),     # 부정
    ("I love this so much!", 1),
    ("Hated it completely!", 0)
]

BERT_MODEL_NAME = 'bert-base-uncased'
model_after = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME, num_labels=2)
from torch.optim import AdamW
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
optimizer = AdamW(model_after.parameters(), lr=5e-5)
model_after.train()
for epoch in range(5):
    for text, label in train_texts:
        inputs = tokenizer(text, return_tensors='pt')
        labels_tensor = torch.tensor([label])
        outputs = model_after(**inputs, labels=labels_tensor)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
model_after.eval()
with torch.no_grad():
    for i ,review in enumerate(test_reviews):
        inputs = tokenizer(review, return_tensors='pt')
        logits = model_after(**inputs).logits
        probs = torch.softmax(logits, dim=-1)[0]
        pred_label = logits.argmax().item()

        print(f'리뷰: {i+1} : {review}')
        print(f'예측 : {labels[pred_label]} {probs[pred_label]:.4f}')
        print(f'긍정 확률 : {probs[1]:.4f}, 부정 확률 : {probs[0]:.4f}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


리뷰: 1 : This movie is absolutely amazing! Best film I've ever seen.
예측 : 긍정 0.8490
긍정 확률 : 0.8490, 부정 확률 : 0.1510
리뷰: 2 : Terrible waste of time. Worst movie ever made.
예측 : 부정 0.8977
긍정 확률 : 0.1023, 부정 확률 : 0.8977
리뷰: 3 : It was okay, nothing special but not bad either.
예측 : 부정 0.6330
긍정 확률 : 0.3670, 부정 확률 : 0.6330
리뷰: 4 : Brilliant acting and stunning visuals throughout!
예측 : 긍정 0.8958
긍정 확률 : 0.8958, 부정 확률 : 0.1042
리뷰: 5 : Boring and predictable plot. Very disappointed.
예측 : 부정 0.8304
긍정 확률 : 0.1696, 부정 확률 : 0.8304


In [13]:
%pip install datasets

  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached xxhash-3.6.0-cp313-cp313-win_amd64.whl.metadata (13 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
Using cached dill-0.4.0-py3-none-any.whl (119 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached anyio-4.11.0-py3-none-any.whl (109 kB)
Using cached xxhash-3.6.0-cp313-cp313-win_amd64.whl (31 kB)

   ----- ---------------------------------- 1/7 [httpcore]
   ----- ---------------------------------- 1/7 [httpcore]
   ----- ---------------------------------- 1/7 [httpcore]
   ----- ---------------------------------- 1/7 [httpcore]
   ----- ---------------------------------- 1/7 [httpcore]
   ----- ---------------------------------- 1/7 [httpcore]
   ----------- ---------------------------- 2/7 [dill]
   


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import warnings
warnings.filterwarnings('ignore')

import nltk
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
import datasets

In [29]:
# 데이터 다운로드 
# 라벨은 pos:1, neg:0
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews
ids = movie_reviews.fileids()
reviews = [movie_reviews.raw(fid) for fid in ids]
labels = [1 if fid.startswith('pos') else 0 for fid in ids]

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\31799\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [38]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    reviews, labels, test_size=0.2, random_state=42
)


In [39]:
# 토크나이저
BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
# 훈련/테스트 데이터 토큰화
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)
len(train_encodings['input_ids']), len(test_encodings['input_ids'])

(1600, 400)

In [41]:
# torch dataset 구성
class MovieReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
train_dataset = MovieReviewsDataset(train_encodings, train_labels)
test_dataset = MovieReviewsDataset(test_encodings, test_labels)
print(f'훈련 데이터셋 크기: {len(train_dataset)}')
print(f'테스트 데이터셋 크기: {len(test_dataset)}')

훈련 데이터셋 크기: 1600
테스트 데이터셋 크기: 400


In [43]:
next(iter(train_dataset))

{'input_ids': tensor([  101,  2096,  3666, 10916,  1010,  2009,  4158,  2000,  2033,  2008,
          6864, 17752,  2121,  2989,  1005,  1055,  2995, 11067,  2004,  1037,
          2143,  1011,  9338,  2003,  9179,  1012,  1999,  3435,  2335,  2012,
          5526,  9629,  2152,  1010,  2016,  2435,  2149,  5977,  9502,  1005,
          1055,  5076, 11867, 11261,  3669,  1025,  1999,  2298,  2040,  1005,
          1055,  3331,  1010,  2016,  2357,  5503, 12688,  2046,  1037,  7968,
          1011, 15729,  3336,  1998,  3024,  2198, 19817, 11431, 27914,  2050,
          2007,  2003,  2034,  2476,  6308,  1025,  1999,  9789,  3238,  1010,
          2016,  2179,  1037,  2732,  4316,  2005,  1996, 23677,  2791,  2008,
          2003,  1006,  2030,  2001,  1007, 15935,  3165,  9221,  1012,  2016,
          3849,  2000,  3305, 16607,  2129,  2000,  2424,  9567,  1996,  4378,
          2097,  2066,  1999,  8741,  1997,  2037, 21407,  1012,  6854,  1010,
          2016,  2089,  2036,  2022,  3

In [46]:
# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME, num_labels=2)
print(f'파라메터수: {sum(p.numel() for p in model.parameters())}')
print(f'학습 가능한 파라메터 : {sum(p.numel() for p in model.parameters() if p.requires_grad)}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


파라메터수: 109483778
학습 가능한 파라메터 : 109483778


In [50]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
# 평가 매트릭스
%pip install evaluate
import evaluate
metric = evaluate.load('accuracy')
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred = logits.argmax(axis=1)
    return metric.compute(predictions=pred, references=labels)


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [55]:
from transformers import (TrainingArguments, Trainer)

In [ ]:
training_args = TrainingArguments(